# Paleo-Detector Analysis Pipeline

This notebook provides a complete, end-to-end workflow for the phenomenological analysis of paleo-detectors. It is designed to calculate the expected nuclear recoil track spectra from both external cosmic ray muon signals and various internal/atmospheric backgrounds.

The pipeline is built around the *mineral_utils.py* module, which contains the *Paleodetector* class. This class acts as the physics engine, handling all low-level data processing and calculations. The notebook serves as a high-level interface to configure and orchestrate the analysis.

### Workflow Overview:

**1. Configuration:** All analysis parameters are defined in the configuration cells. This includes selecting the mineral from a library, defining the geological history of the sample (age, exposure time, deposition rate), and setting up the astrophysical scenarios for the cosmic ray flux.

**2. Initialization:** A Paleodetector object is created. This object automatically loads and caches all necessary input data, such as SRIM tables for ion ranges and pre-computed Geant4 results for muon interactions.

**3. Background Calculation:** The notebook calculates the track spectra for the primary background sources:

- Spontaneous Fission: Tracks from the spontaneous fission of Uranium-238 impurities within the mineral.

- Radiogenic Neutrons: Tracks from neutrons produced by (α,n) reactions in the surrounding rock.

- Neutrinos: Tracks from coherent elastic neutrino-nucleus scattering.

**7. Signal Calculation:** For each defined astrophysical scenario, the notebook calculates the track spectrum from cosmic ray muons. This core step includes a sophisticated model for a time-variant cosmic ray flux and the continuous deposition of overburden, which attenuates the signal over the exposure window. The time integration is performed in parallel for efficiency.

**8. Analysis & Plotting:** The final signal and background components are combined to produce the summary plots, showing the total expected number of tracks as a function of track length.


In [ ]:
# --- Core Libraries ---
import numpy as np
from matplotlib import pyplot as plt

# --- Custom Utility Module ---
from mineral_utils import Paleodetector

print("Libraries imported successfully.")

In [ ]:
CONFIG = {
    # --- Mineral and File Configuration ---
    "mineral_name": "Olivine",  # Must match a key in MINERAL_LIBRARY below
    
    # --- Geological & Sample Parameters ---
    "sample_mass_kg": 0.0002,       # Sample mass in kg (e.g., 0.01 for 10g)
    "total_age_myr" : 0.04,

    "total_exposure_kyr": 40.1,         # Total age of the sample in kyr
    "exposure_window_kyr": 40.,  # Duration of CR signal exposure in kyr

    # --- Plotting & Binning Configuration ---
    "n_bins": 60,
    "x_min_log_nm": 2.5, # Min track length in log10(nm)
    "x_max_log_nm": 5., # Max track length in log10(nm)

    # Parameters derived from Geant4 simulation setup
    "geant4_energy_bins_gev": [0.01      , 0.01165914, 0.01359356, 0.01584893, 0.0184785 ,
        0.02154435, 0.02511886, 0.02928645, 0.03414549, 0.03981072,
        0.04641589, 0.05411695, 0.06309573, 0.07356423, 0.08576959,
        0.1       , 0.11659144, 0.13593564, 0.15848932, 0.18478498,
        0.21544347, 0.25118864, 0.29286446, 0.34145489, 0.39810717,
        0.46415888, 0.54116953, 0.63095734, 0.73564225, 0.8576959 ,
        1.0000, 1.1909, 1.4183, 1.6891, 2.0116, 2.3956, 2.8530, 3.3977,
        4.0464, 4.8189, 5.7390, 6.8347, 8.1396, 9.6936, 11.5443, 13.7484, 
        16.3732, 19.4993, 23.2221, 27.6557, 32.9358, 39.2240, 46.7127, 
        55.6312, 66.2524, 78.9015, 93.9656, 111.9057, 133.2710, 158.7153, 
        189.0176, 225.1053, 268.0829, 319.2659, 380.2209, 452.8135, 539.2657, 
        642.2235, 764.8383, 910.8629, 1084.7669, 1291.8731, 1538.5204, 1832.2582, 
        2182.0771, 2598.6842, 3094.8308, 3685.7029, 4389.3856, 5227.4170, 
        6225.4472, 7414.0236, 8829.5257, 10515.2787, 12522.8794, 14913.7758, 
        17761.1475, 21152.1459, 25190.5612, 30000.0000, 100000],

    "mineral_target_thickness_cm": 1000, # Thickness of the target in the Geant4 simulation in cm
    "total_simulated_muons": 1e5,        # Number of muons simulated in each Geant4 run, for weighting
}

In [ ]:
scenario_config_simple = {
    'name': 'simple',
    'event_fluxes': {
        0.: ('H3a', 'H3a'),
    }
}

scenario_config_simple_enhanced = {
    'name': 'simple_enhanced',
    'event_fluxes': {
        0.: ('H3a_enhanced', 'H3a_enhanced'),
        5.: ('H3a', 'H3a'),
    }
}

scenario_config_SN250_enhanced = {
    'name': 'SN250_enhanced',
    'event_fluxes': {
        0.: ('SN250pc_10kyr_enhanced', 'SN250pc_10kyr_enhanced'),
        2.: ('SN250pc_12kyr_enhanced', 'SN250pc_12kyr_enhanced'),
        5.: ('SN250pc_15kyr_enhanced', 'SN250pc_15kyr_enhanced'),
        10.: ('SN250pc_20kyr', 'SN250pc_20kyr'),
        20.: ('SN250pc_30kyr', 'SN250pc_30kyr'),
        25.: ('SN250pc_35kyr', 'SN250pc_35kyr'),
        30.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        35.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        40.: ('H3a', 'H3a'),
    }
}

scenario_config_SN250 = [{
    'name': 'SN250_40',
    'event_fluxes': {
        0.: ('SN250pc_10kyr', 'SN250pc_10kyr'),
        10.: ('SN250pc_20kyr', 'SN250pc_20kyr'),
        20.: ('SN250pc_30kyr', 'SN250pc_30kyr'),
        30.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        40.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_30',
    'event_fluxes': {
        0.: ('SN250pc_20kyr', 'SN250pc_20kyr'),
        10.: ('SN250pc_30kyr', 'SN250pc_30kyr'),
        15.: ('SN250pc_35kyr', 'SN250pc_35kyr'),
        20.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        25.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        30.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_13',
    'event_fluxes': {
        0.: ('SN250pc_35kyr', 'SN250pc_35kyr'),
        3.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        8.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        13.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_11',
    'event_fluxes': {
        0.: ('SN250pc_35kyr', 'SN250pc_35kyr'),
        1.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        6.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        11.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_10',
    'event_fluxes': {
        0.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        5.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        10.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_8',
    'event_fluxes': {
        0.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        3.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        8.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_7',
    'event_fluxes': {
        0.: ('SN250pc_40kyr', 'SN250pc_40kyr'),
        2.: ('SN250pc_45kyr', 'SN250pc_45kyr'),
        7.: ('H3a', 'H3a'),
    }
},
{
    'name': 'SN250_0',
    'event_fluxes': {
        0.: ('H3a', 'H3a'),
    }
},
]

In [ ]:
# --- Mineral Library ---
# To analyze a new mineral, add its properties here and ensure the
# corresponding SRIM and Geant4 data files exist in the Data/ directory.
MINERAL_LIBRARY = {
    "Halite": {
        "name": "Halite",
        "shortname": "Hal",
        "composition": "Na-Cl",
        "nuclei": ["Na", "Cl"],
        "stoich": [1, 1],
        "uranium_concentration_g_g": 1e-9,
        "density_g_cm3": 2.16,
    },
    "Olivine": {
        "name": "Olivine",
        "shortname": "Oli",
        "composition": "Mg-Fe-Si- O",
        "nuclei": ["Mg", "Fe", "Si", "O"],
        "stoich": [1.8, 0.2, 1, 4],
        "uranium_concentration_g_g": 1e-7,
        "density_g_cm3": 3.8,
    },
    "Quartz": {
        "name": "Quartz",
        "shortname": "Qz",
        "composition": "Si- O",
        "nuclei": ["Si", "O"],
        "stoich": [1, 2],
        "uranium_concentration_g_g": 1e-5,
        "density_g_cm3": 2.6,
    },
    "Spinel": {
        "name": "Spinel",
        "shortname": "Spi",
        "composition": "Mg-Al- O",
        "nuclei": ["Mg", "Al", "O"],
        "stoich": [1, 2, 4],
        "uranium_concentration_g_g": 1e-7,
        "density_g_cm3": 3.7,
    }
}

In [ ]:
x_bins2 = np.logspace(CONFIG["x_min_log_nm"], CONFIG["x_max_log_nm"], CONFIG["n_bins"])
x_bins = np.linspace(0, 100000, 100)
x_mids = x_bins[:-1] + np.diff(x_bins) / 2
x_mids2 = x_bins2[:-1] + np.diff(x_bins2) / 2

In [ ]:
exposure_times = np.array([41.4, 30., 13.1, 10.96, 9.5, 8.64, 7.65, 0.005])
exposure_times_err = np.array([1., 2., 0.7, 0.15, 0.5, 0.06, 0.12, 0.0001])
names = np.array(["Laschamp", "Lemptegy", "Come", "Dome", "Pariou", "Lavache", "Montcineyre", "Modern"], dtype=str)


In [ ]:
# --- Setup & Verification ---
mineral_config = MINERAL_LIBRARY.get(CONFIG["mineral_name"])
if not mineral_config:
    raise ValueError(f"Mineral '{CONFIG['mineral_name']}' not found in MINERAL_LIBRARY.")

mineral = Paleodetector(mineral_config)

## Calculation of Track Spectra

With the Paleodetector object initialized and all necessary data loaded, we can now proceed with the core physics calculations. The following cells will compute the differential track rate (dR/dx) for each relevant physical process and then integrate these rates over the sample's lifetime to find the total expected number of tracks in each track length bin.

### Calculation Steps:

**1. Backgrounds:** We first calculate the spectra for the steady-state backgrounds that accumulate over the entire geological age of the sample. These include spontaneous fission, radiogenic neutrons, and atmospheric neutrinos.

**2. Muon Signal:** Next, we compute the primary signal from cosmic ray muons. The integrate_muon_signal_spectrum_parallel function is used for this. It performs a numerical integration over the specified exposure_window_kyr, accounting for two key time-dependent effects at each step:

- The evolution of the cosmic ray flux, which is interpolated from the files defined in the scenario_config.

- The increase in shielding due to the continuous deposition_rate_m_kyr.

**3. Final Plot:** Finally, all calculated track components (signals and backgrounds) are summed and plotted to visualize the expected final track spectrum in the detector.

In [ ]:
total_nu_tracks = np.array([mineral.integrate_nu_spectrum(x_bins, age/1000., CONFIG['sample_mass_kg'], flux_name="all") for age in exposure_times])

In [ ]:
total_fission_tracks = np.array([mineral.integrate_fission_spectrum(x_bins, age/1000., CONFIG['sample_mass_kg']) for age in exposure_times])

In [ ]:
total_neutron_tracks = np.array([mineral.integrate_neutron_spectrum(x_bins, age/1000., CONFIG['sample_mass_kg']) for age in exposure_times])

In [ ]:
total_muon_tracks = np.array([mineral.integrate_muon_signal_spectrum_parallel(x_bins, scenario_config=scenario_config_simple, energy_bins_gev=CONFIG["geant4_energy_bins_gev"], exposure_window_kyr=age, sample_mass_kg = CONFIG['sample_mass_kg'], nsteps=5) for age in exposure_times])

In [ ]:
total_muon_tracks_SN250 = np.array([mineral.integrate_muon_signal_spectrum_parallel(x_bins, scenario_config=scenario_config_SN250[i], energy_bins_gev=CONFIG["geant4_energy_bins_gev"], exposure_window_kyr=age, sample_mass_kg = CONFIG['sample_mass_kg'], nsteps=int(age)) for i, age in enumerate(exposure_times)])

In [ ]:
muon_tracks_enhanced = mineral.integrate_muon_signal_spectrum_parallel(x_bins, scenario_config=scenario_config_simple_enhanced, energy_bins_gev=CONFIG["geant4_energy_bins_gev"], exposure_window_kyr=exposure_times[0], sample_mass_kg = CONFIG['sample_mass_kg'], nsteps=41)
muon_tracks_SN250_enhanced = mineral.integrate_muon_signal_spectrum_parallel(x_bins, scenario_config=scenario_config_SN250_enhanced, energy_bins_gev=CONFIG["geant4_energy_bins_gev"], exposure_window_kyr=exposure_times[0], sample_mass_kg = CONFIG['sample_mass_kg'], nsteps=41)

In [ ]:
total_muon_tracks_enhanced = total_muon_tracks.copy()
total_muon_tracks_SN250_enhanced = total_muon_tracks_SN250.copy()

total_muon_tracks_enhanced[0] = muon_tracks_enhanced
total_muon_tracks_SN250_enhanced[0] = muon_tracks_SN250_enhanced


In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])

for i, name in enumerate(names):
    ax.plot(x_mids/1000, total_fission_tracks[i]/(1e3*CONFIG['sample_mass_kg']), color=f'C{i+3}', linestyle='dashed', linewidth=0.8)
    ax.plot(x_mids/1000, total_neutron_tracks[i]/(1e3*CONFIG['sample_mass_kg']), color=f'C{i+3}', linestyle='dotted', linewidth=0.8)
    ax.plot(x_mids/1000, total_muon_tracks[i]/(1e3*CONFIG['sample_mass_kg']), color=f'C{i+3}', label=f"  Muon-induced tracks: {names[i]}")


ax.set_title(f"Track density distribution in {CONFIG['mineral_name']}")
ax.set_xlabel(r"Track Length ($\mu$m)")
ax.set_ylabel(r"Density of Tracks (g$^{-1}$)")
ax.set_yscale("log")
ax.set_ylim(1e-2, 1e7)
ax.set_xlim(0, 50)
ax.legend(fontsize='small')
plt.savefig("Plots/CDP_track_number_per_gram.png", dpi=500)
plt.show()


In [ ]:
summed_target = np.sum(total_muon_tracks[..., 15:], axis=-1)
summed_SN250 = np.sum(total_muon_tracks_SN250[..., 15:], axis=-1)
summed_enhanced = np.sum(total_muon_tracks_enhanced[..., 15:], axis=-1)
summed_SN250_enhanced = np.sum(total_muon_tracks_SN250_enhanced[..., 15:], axis=-1)


In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])


for i, name in enumerate(names):
    ax.plot(x_mids/1000, total_muon_tracks[i], linestyle='--', color=f'C{i+3}',linewidth=0.7)

sum_muons = np.sum(total_muon_tracks, axis=0)
sum_muons_SN250 = np.sum(total_muon_tracks_SN250, axis=0)


ax.plot(x_mids/1000, sum_muons, label=f"Sum of muon-induced tracks", linewidth=1.7, color='C0')
ax.plot(x_mids/1000, sum_muons_SN250, label=f"Sum of muon-induced tracks (with SN @ 250 pc)", linewidth=1.7, color='C1')
ax.plot(x_mids/1000, np.sum(total_fission_tracks, axis=0), label=f"Sum of fission-induced tracks", color='green', linewidth=1.7)
ax.fill_between(x_mids/1000, sum_muons-np.sqrt(sum_muons), sum_muons+np.sqrt(sum_muons), color='C0', alpha=0.3)
ax.fill_between(x_mids/1000, sum_muons_SN250-np.sqrt(sum_muons_SN250), sum_muons_SN250+np.sqrt(sum_muons_SN250), color='C1', alpha=0.3)

ax.fill_between(x_mids/1000, 1e-3, 1, color='grey', alpha=0.15)



ax.set_title(f"Total track count distribution in {CONFIG['mineral_name']}")
ax.set_xlabel(r"Track Length ($\mu$m)")
ax.set_ylabel(r"Number of tracks in 8*0.2 g samples")
ax.set_yscale("log")
ax.set_ylim(1e-2, 3e3)
ax.set_xlim(5, 45)
ax.legend()
plt.savefig("Plots/CDP_total_tracks_collection.png", dpi=500)
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

fig, ax = plt.subplots(figsize=[7, 5])


ax.errorbar(exposure_times, summed_target, yerr= np.sqrt(summed_target), label="Normal scenario", linewidth=1.8, color='C0', marker='o', markersize=1., linestyle='--', capsize=0)
ax.fill_between(exposure_times, summed_target-np.sqrt(summed_target), summed_target+np.sqrt(summed_target), color='C0', alpha=0.7)
ax.fill_between(exposure_times, summed_target-np.sqrt(summed_target+(0.1*summed_target)**2), summed_target+np.sqrt(summed_target+(0.1*summed_target)**2), color='C0', alpha=0.3)
ax.fill_between(exposure_times, summed_target-np.sqrt(summed_target+(0.3*summed_target)**2), summed_target+np.sqrt(summed_target+(0.3*summed_target)**2), color='C0', alpha=0.2)

ax.errorbar(exposure_times, summed_SN250, yerr= np.sqrt(summed_SN250), label="With SN @ 250 pc", linewidth=1.8, color='C1', marker='o', markersize=1., linestyle='--', capsize=0)
ax.fill_between(exposure_times, summed_SN250-np.sqrt(summed_SN250), summed_SN250+np.sqrt(summed_SN250), color='C1', alpha=0.7)
ax.fill_between(exposure_times, summed_SN250-np.sqrt(summed_SN250+(0.1*summed_SN250)**2), summed_SN250+np.sqrt(summed_SN250+(0.1*summed_SN250)**2), color='C1', alpha=0.3)
ax.fill_between(exposure_times, summed_SN250-np.sqrt(summed_SN250+(0.3*summed_SN250)**2), summed_SN250+np.sqrt(summed_SN250+(0.3*summed_SN250)**2), color='C1', alpha=0.2)

ax.errorbar(exposure_times, summed_enhanced, yerr= np.sqrt(summed_enhanced), label="Normal + Laschamp enhancement", linewidth=1., color='C0', marker='o', markersize=0.8, linestyle='-.', capsize=0)
ax.errorbar(exposure_times, summed_SN250_enhanced, yerr= np.sqrt(summed_SN250_enhanced), label="SN + Laschamp enhancement", linewidth=1., color='C1', marker='o', markersize=0.8, linestyle='-.', capsize=0)

for i, age in enumerate(exposure_times):
    ax.axvspan(age-exposure_times_err[i], age+exposure_times_err[i], color='grey', alpha=0.4)

axins = zoomed_inset_axes(ax, zoom=3., loc='upper left',bbox_to_anchor=(0.065, 0.6, 0.4, 0.4), bbox_transform=ax.transAxes)
axins.errorbar(exposure_times, summed_target, yerr= np.sqrt(summed_target), label="Normal scenario", linewidth=1.8, color='C0', marker='o', markersize=1., linestyle='--', capsize=0)
axins.fill_between(exposure_times, summed_target-np.sqrt(summed_target), summed_target+np.sqrt(summed_target), color='C0', alpha=0.7)
axins.fill_between(exposure_times, summed_target-np.sqrt(summed_target+(0.1*summed_target)**2), summed_target+np.sqrt(summed_target+(0.1*summed_target)**2), color='C0', alpha=0.3)
axins.fill_between(exposure_times, summed_target-np.sqrt(summed_target+(0.3*summed_target)**2), summed_target+np.sqrt(summed_target+(0.3*summed_target)**2), color='C0', alpha=0.2)

axins.errorbar(exposure_times, summed_SN250, yerr= np.sqrt(summed_SN250), label="With SN @ 250 pc", linewidth=1.8, color='C1', marker='o', markersize=1., linestyle='--', capsize=0)
axins.fill_between(exposure_times, summed_SN250-np.sqrt(summed_SN250), summed_SN250+np.sqrt(summed_SN250), color='C1', alpha=0.7)
axins.fill_between(exposure_times, summed_SN250-np.sqrt(summed_SN250+(0.1*summed_SN250)**2), summed_SN250+np.sqrt(summed_SN250+(0.1*summed_SN250)**2), color='C1', alpha=0.3)
axins.fill_between(exposure_times, summed_SN250-np.sqrt(summed_SN250+(0.3*summed_SN250)**2), summed_SN250+np.sqrt(summed_SN250+(0.3*summed_SN250)**2), color='C1', alpha=0.2)

for i, age in enumerate(exposure_times):
    axins.axvspan(age-exposure_times_err[i], age+exposure_times_err[i], color='grey', alpha=0.4)


axins.set_xlim(7, 14)
axins.set_ylim(150, 1000)
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5", linestyle='--')

ax.set_xlabel("Exposure Time (kyr)")
ax.set_ylabel(r"Number of tracks >5 $\mu$m in 0.2 g sample")
ax.set_title("Time Evolution of Muon-Induced Tracks")
ax.set_ylim(1e0, 4.5e3)
ax.legend(fontsize='small', loc='upper right')
plt.savefig("Plots/CDP_time_evolution.png", dpi=500)
plt.show()

In [ ]:
filepath = './Data/processed_recoils/Olivine_muon_recoil_simple_10.0kyr_0mwe.npz'
recoil_data = np.load(filepath)

drdx_muon = mineral._convert_recoil_to_track_spectrum(x_bins=x_bins2, recoil_data=recoil_data, energy_bins_gev=CONFIG["geant4_energy_bins_gev"])
drdx_fission = mineral.calculate_fission_spectrum(x_bins2)
drdx_neutron = mineral.calculate_neutron_spectrum(x_bins2)

drdx_org = np.zeros_like(drdx_muon['total'])
drdx_result = np.zeros_like(drdx_muon['total'])
for name in recoil_data.files[1:]:
    if name != 'total' and name != 'Er_bins':
        if name in ['Mg', 'Fe', 'Si', 'O']:
            drdx_org += drdx_muon[name]
        else: drdx_result += drdx_muon[name]

In [ ]:
filepath_mod = './Data/processed_recoils/Olivine_muon_recoil_SN250_40_0.0kyr_0mwe.npz'
recoil_data_mod = np.load(filepath_mod)

drdx_muon_mod = mineral._convert_recoil_to_track_spectrum(x_bins=x_bins2, recoil_data=recoil_data_mod, energy_bins_gev=CONFIG["geant4_energy_bins_gev"])

drdx_org_mod = np.zeros_like(drdx_muon['total'])
drdx_result_mod = np.zeros_like(drdx_muon['total'])
for name in recoil_data_mod.files[1:]:
    if name != 'total' and name != 'Er_bins':
        if name in ['Mg', 'Fe', 'Si', 'O']:
            drdx_org_mod += drdx_muon_mod[name]
        else: drdx_result_mod += drdx_muon_mod[name]

In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])

ax.plot(x_mids2, drdx_muon['total'], label='Total muon-induced')
ax.plot(x_mids2, drdx_muon_mod['total'], linestyle='--', linewidth=1., label='Total muon-induced at peak SN')
ax.plot(x_mids2, drdx_org, linestyle='-.', linewidth=0.6, color='C5', label='Target nuclides only')
ax.plot(x_mids2, drdx_result, linestyle='-.', linewidth=0.6, color='C6', label='Nuclides from spallation')
ax.plot(x_mids2, drdx_neutron, linestyle='-', color = 'C4',label='Neutron-induced')
ax.plot(x_mids2, drdx_fission, linestyle='-', color = 'C2',label='Fission-induced')

ax.set_yscale("log")
ax.set_xscale("log")
ax.set_xlabel("Track Length (nm)")
ax.set_ylabel(r"dR/dx (nm$^{-1}$ kg$^{-1}$ Myr$^{-1}$)")
ax.set_title(f"Track rate spectrum in {CONFIG['mineral_name']}")
ax.set_ylim(1e-1, 3e7)
ax.legend()
plt.savefig("Plots/CDP_drdx.png", dpi=500)
plt.show()

